### Проверка того, что наше api работает

Перед началом необходимо собрать docker-образ и запустить контейнер

1. Следовать инструкциям https://github.com/fimochka-sudo/GB_docker_flask_example/blob/main/README.md
2. Далее уже запускать код ниже

In [3]:
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve
from urllib import request, parse

X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

C:\Users\Admin\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Admin\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\Admin\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [4]:
X_test[['description', 'company_profile', 'benefits']].head(3)

,description,company_profile,benefits
0,Stylect is a dynamic startup that helps helps ...,NaN,We are negotiable on salary and there is the p...
1,General Summary: Achieves maximum sales profit...,NaN,Great Health and DentalFast Advancement Opport...
2,"At ustwo™ you get to be yourself, whilst deliv...",ustwo offers you the opportunity to be yoursel...,NaN


In [5]:
import urllib.request
import json      

def get_prediction(description, company_profile, benefits):
    body = {'description': description, 
                            'company_profile': company_profile,
                            'benefits': benefits} 

    myurl = "http://127.0.0.1:5000/predict"
#     myurl = "http://127.0.0.1:8180/predict"
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    #print (jsondataasbytes)
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']

In [9]:
get_prediction('Stylect is a dynamic startup that h', 'offers', 'free ')

0.10692754073840159

In [24]:
%%time
predictions = X_test[['description', 'company_profile', 'benefits']].iloc[:500].apply(lambda x: get_prediction(x[0], 
                                                                                               x[1], 
                                                                                               x[2]), 1)

Wall time: 14.9 s


In [8]:
roc_auc_score(y_score=predictions.values, y_true=y_test.iloc[:500])

0.9803161348046525

In [7]:
#X_test[['description', 'company_profile', 'benefits']].head(3)

Давайте подумаем, где наше api может сломаться

In [15]:
get_prediction("", "", None)

0.13518841847988364